In [ ]:
import ssl
import os
import pandas as pd
import certifi
import requests

from bs4 import BeautifulSoup
from selenium import webdriver
from pymongo import MongoClient

In [2]:
ssl._create_default_https_context = ssl._create_unverified_context

In [9]:
uri = os.environ['MONGODB_CONNECT_STRING']
client = MongoClient(uri, tlsCAFile=certifi.where())

db = client['ai-test']
collection = db['test']

In [3]:
def get_url_images(url):

    driver = webdriver.Chrome()
    driver.get(url)

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    images = soup.find_all('img')

    if not os.path.exists('airbnb_photos'):
        os.makedirs('airbnb_photos')

    for i, img in enumerate(images):
        try:
            img_url = img['src']
            img_data = requests.get(img_url).content
            with open(f'airbnb_photos/photo_{i}.jpg', 'wb') as handler:
                handler.write(img_data)
            print(f'Downloaded: photo_{i}.jpg')
        except KeyError:
            continue

    for img in images:
        print(img['src'])
    driver.quit()

    return [img['src'] for img in images]


In [ ]:
url = "https://www.airbnb.com/rooms/6624170"
img_urls = get_url_images(url)
print(get_url_images(url))
# collection.insert_one({ 'appartment_id': 6624170, 'img_urls': img_urls })

['https://a0.muscache.com/im/pictures/df91da10-f7d4-4cf8-80b6-8add6047c989.jpg?im_w=720', 'https://a0.muscache.com/im/pictures/0a464ad0-9a98-4e10-b33f-43892b569293.jpg?im_w=720', 'https://a0.muscache.com/im/pictures/f2558d9f-6fc1-423d-a94a-eb016fa25e32.jpg?im_w=720', 'https://a0.muscache.com/im/pictures/bb99f545-90db-449e-a48b-cf157fd5475c.jpg?im_w=720', 'https://a0.muscache.com/im/pictures/debf81e2-4d62-4fab-ac93-732d996e4907.jpg?im_w=720']


InsertOneResult(ObjectId('6713c681a83e125c903263b2'), acknowledged=True)

In [5]:
# csvfile_path = "./Amsterdam/listings_cleaned.csv"
csvfile_path = "./NewYork/listings_cleaned.csv"
listings_csv_data = pd.read_csv(csvfile_path)[:10]
img_lists = []

for index, row in listings_csv_data.iterrows():
    url = row["listing_url"]
    listing_id = row["id"]
    images = get_url_images(url)
    img_lists.append({ 'appartment_id': listing_id, 'img_urls': images })

Downloaded: photo_0.jpg
Downloaded: photo_1.jpg
Downloaded: photo_2.jpg
Downloaded: photo_3.jpg
Downloaded: photo_4.jpg
https://a0.muscache.com/im/pictures/5f44a178-6043-449b-9d9f-e0014111fe4c.jpg?im_w=720&im_format=avif
https://a0.muscache.com/im/pictures/e3a14909-b4e0-457b-84f2-5ff9e643e32a.jpg?im_w=720&im_format=avif
https://a0.muscache.com/im/pictures/miso/Hosting-572612125615500056/original/e24d40af-2dc8-4532-af0b-0f40b6e3dd57.jpeg?im_w=720&im_format=avif
https://a0.muscache.com/im/pictures/miso/Hosting-572612125615500056/original/88bf45a5-2936-4bfe-8c65-21f2121166a3.jpeg?im_w=720&im_format=avif
https://a0.muscache.com/im/pictures/cd2abf18-11e8-4d77-8c76-7d4157ac47e3.jpg?im_w=720&im_format=avif
Downloaded: photo_0.jpg
Downloaded: photo_1.jpg
Downloaded: photo_2.jpg
Downloaded: photo_3.jpg
Downloaded: photo_4.jpg
https://a0.muscache.com/im/pictures/011da841-7234-42b9-9865-517a9be34127.jpg?im_w=720
https://a0.muscache.com/im/pictures/2fbecb34-36a8-4456-bcba-36daaea91763.jpg?im_w=720

In [ ]:
collection.insert_many(img_lists)